# Apache Spark - Testing Streaming

<p><strong>Objetivo: </strong> El objetivo de este cuaderno es utilizar la API de Dataframe para crear un ejemplo de uso de streaming estructurado:</p>

<p><strong>Nota: </strong> Para ejecutar este notebook es necesario utilizar versión 5.5 de Spark. Posteriores no permiten creación de archivos.</p>

##Generando datos para consumo

En este ejemplo se estarán generando datos ejemplos en formato JSON para luego consumirlos en forma de streaming. El primer paso a seguir es crear una carpeta <b>streamtst</b> para depositar los archivos. Pueden crear la carpeta directamente sobre FileStore:

In [0]:
dbutils.fs.mkdirs('/FileStore/streamtst')

Out[1]: True

Para generar los datos de prueba se crea un diccionario simple con dos columnas. El primero, <b>unix_timestamp</b>, se completa con el tiempo actual. El segundo se llena con un número aleatorio <b>random_number</b>. Se convierte el resultado a formato JSON y se ubica el resultado en la carpeta <b>streamtst</b>. El comando del sistema de archivos enumerará todo en la carpeta streamtst. Solo debería haber un archivo en la carpeta, 0.json. Cada archivo tendrá un marca de tiempo y un número aleatorio. Por el momento solo se creará un archivo:

In [0]:
import json
import random
import time
d = {'unix_timestamp':time.time(),'random_number':random.randint(1,10)}
with open("/dbfs/FileStore/streamtst/0.json", "w") as f:
  json.dump(d, f)

In [0]:
%fs ls /FileStore/streamtst/

path,name,size
dbfs:/FileStore/streamtst/0.json,0.json,58


##Configurar el stream

A continuacion se va a definir el esquema para que Databricks sepa con qué se está trabajado y sea más fácil el procesamiento:

In [0]:
from pyspark.sql.types import TimestampType, IntegerType, StructType, StructField
schema = StructType([
StructField("unix_timestamp", TimestampType(), True),
StructField("random_number", IntegerType(), True) ])

En este paso se puede definir el <b>readStream</b>, que se asigna el esquema, se establece que se leerá un archivo en cada trigger(por defecto son 1000), el formato y localización de los datos. Solo definimos la configuración, todavía no ocurre nada:

In [0]:
streamingDF = (
  spark
    .readStream
    .schema(schema)
    .option('maxFilesPerTrigger',1)
    .json('/FileStore/streamtst/')
)

En este paso se configura una consulta de una agregación simple donde se cuenta el número de ocurrencias de cada número aleatorio:

In [0]:
streamingDFAgg = (streamingDF.groupBy(streamingDF.random_number).count())

##Iniciar el flujo stream

Una vez que está todo definido se puede iniciar el stream. Se creará una tabla llamada <b>ramdom_numbers</b> y se va a depositar en la memoria. El parámetro <b>outputMode</b> define lo que se enviará al llamado sumidero o destino (en este caso, la memoria), se puede elegir entre Append, Update y Complete. Cómo se va a realizar una Agregación, es necesario utilizar Complete:

In [0]:
dfrun = (
  streamingDFAgg
    .writeStream
    .format("memory")
    .queryName("random_numbers")
    .outputMode("complete")
    .start()
)

Ejecute la siguiente consulta para visualizar la cantidad de números que se obtuvieron de los datos. Configure un gráfico con <b>KEYS=random_number</b> y <b>VALUES=count</b>:

In [0]:
%sql select * from random_numbers;

random_number,count
8,1


El proceso de streaming que se ha configurado solo ha leido un archivo, el que está en la carpeta actual. Se van a agregar más archivos para ver el proceso en funcionamiento. Para ellos ejecute el siguiente código que ubicará más archivos en la carpeta <b>streamtst</b>:

In [0]:
for i in range(1,100):
  d = {'unix_timestamp':time.time()
    ,'random_number':random.randint(1,10)}
  with open('/dbfs/FileStore/streamtst/{}.json'.format(i), 'w') as f:
    json.dump(d, f)

Ahora puede visualizar que hay muchos más archivos dentro de la carpeta, y el proceso de streaming comenzará a leerlos:

In [0]:
%fs ls /FileStore/streamtst/

path,name,size
dbfs:/FileStore/streamtst/0.json,0.json,58
dbfs:/FileStore/streamtst/1.json,1.json,58
dbfs:/FileStore/streamtst/10.json,10.json,57
dbfs:/FileStore/streamtst/11.json,11.json,58
dbfs:/FileStore/streamtst/12.json,12.json,57
dbfs:/FileStore/streamtst/13.json,13.json,57
dbfs:/FileStore/streamtst/14.json,14.json,59
dbfs:/FileStore/streamtst/15.json,15.json,57
dbfs:/FileStore/streamtst/16.json,16.json,58
dbfs:/FileStore/streamtst/17.json,17.json,57


Ejecute nuevamente la consulta anterior para ver los nuevos resultados y cómo se van agregando datos a los resultados calculados previamente:

In [0]:
%sql select * from random_numbers ORDER BY random_number;

random_number,count
7,1
8,1


In [0]:
%sql select * from random_numbers ORDER BY random_number;

random_number,count
4,1
5,1
7,1
8,1


##Chequeando el estado del Stream

Si bien es bueno tener el proceso de streaming en ejecución si obtiene datos, mantiene el clúster en funcionamiento. Por lo tanto, puede resultar costoso si no se tiene cuidado. Si desea ver si se está ejecutando algún streaming, puede hacerse ejecutando este código. Hay un flujo de streaming en ejecución:

In [0]:
for stream in spark.streams.active:
  print("{}, {}".format(stream.name, stream.id))

random_numbers, 3b1e08eb-66d3-4542-847c-af55f12b43f8

Para detener el flujo debe utilizar el siguiente comando:

In [0]:
dfrun.stop()

Si desea detener todos los streams activos, puede ejecutar otra versión del comando anterior, cuidado de no detener nada por error:

In [0]:
for stream in spark.streams.active:
  stream.stop()

Un detalle interesante es que la tabla que configuramos sigue activa. Ya no se actualizará, pero puede consultarla siempre que no detenga el clúster ni lo elimine activamente.

##Ejecutando un poco más rápido

En el ejemplo anterior limitamos la velocidad a un archivo por Trigger. Eso no es lo que se hace normalmente. Usualmente se deja que Databricks ejecute a un ritmo superior. Lo que se obtiene sin ningún ajuste es 1000. Intentémoslo. Se configuran nuevamente los Dataframes, pero ahora no se pone el límite de 1 fichero por Trigger:

In [0]:
streamingDF = (spark.readStream.schema(schema).option('maxFilesPerTrigger',50).json('/FileStore/streamtst/'))
streamingDFAgg = (streamingDF.groupBy(streamingDF.random_number).count())

dfrun = (
  streamingDFAgg
    .writeStream
    .format("memory")
    .queryName("random_numbers")
    .outputMode("complete")
    .start()
)

Ejecute la consulta SQL nuevamente y compruebe que los resultados aparecen mucho más rápido:

In [0]:
%sql select * from random_numbers ORDER BY random_number;

random_number,count
1,7
2,9
3,2
4,6
5,6
6,4
7,4
8,5
9,1
10,6


##Limpiando el espacio de trabajo

Si desea eliminar la carpeta y los archivos creados en el DBFS puede ejecutar el siguiente comando:

In [0]:
dbutils.fs.rm("/FileStore/streamtst/",True)